In [1]:
# 导入相关库
import pandas as pd
from pyecharts import Sankey
pd.set_option('display.max_rows', 500)

ERROR:lml.utils:failed to import pyecharts_snapshot
Traceback (most recent call last):
  File "d:\python\virtualenvs\jupy36\lib\site-packages\lml\utils.py", line 43, in do_import
    plugin_module = __import__(plugin_module_name)
ModuleNotFoundError: No module named 'pyecharts_snapshot'


In [2]:
# data = pd.read_csv(r"D:\GitHub\datasets\YTX\info\data_main_10k1gai2.csv")
data = pd.read_excel("D:/GitHub/datasets/YTX/汇总表/9月各线路进出站流量汇总.xlsx")

In [3]:
data.shape

(447, 3)

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 447 entries, 0 to 446
Data columns (total 3 columns):
entry_line_id    447 non-null int64
exit_line_id     447 non-null int64
jishu            447 non-null int64
dtypes: int64(3)
memory usage: 10.6 KB


#### 提取数据

In [5]:
# df = data[["entry_station_name","entry_line","entry_所在区","exit_line","exit_所在区","exit_station_name","status"]]
df = data.copy()

In [6]:
df["entry_line_id"] = df["entry_line_id"].map(lambda x:str(x))
df["exit_line_id"] = df["exit_line_id"].map(lambda x:str(x))

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 447 entries, 0 to 446
Data columns (total 3 columns):
entry_line_id    447 non-null object
exit_line_id     447 non-null object
jishu            447 non-null int64
dtypes: int64(1), object(2)
memory usage: 10.6+ KB


In [8]:
dict_stations = {"1":"1号线","2":"2号线","4":"4号线","5":"5号线","6":"6号线","7":"7号线","8":"8号线",
"9":"9号线","10":"10号线","13":"13号线","14":"14号线","15":"15号线","16":"16号线","90":"10号线",
"91":"S1线","92":"燕房线","93":"大兴线","94":"昌平线","95":"房山线","96":"亦庄线","97":"八通线",
"98":"首都机场线","83":"S2线","84":"怀柔密云线","85":"副中心线",
"88":"大兴机场线","89":"西郊线"}
group_commuting_start = df.replace(dict_stations)
# dict_stations ={"10号线二期发行中心":"10号线","14号线发行中心":"14号线","8号线（奥运专线）":"8号线"}
# df = df.replace(dict_stations)

In [9]:
# df_commuting_start = df[df["status"] == "通勤出发"].reset_index(drop=True)
# df_commuting_start.shape

In [10]:
# df_commuting_start[(df_commuting_start["entry_line"] == "10号线") & (df_commuting_start["exit_line"] == "S1线")]

In [11]:
# group_commuting_start = df_commuting_start.groupby(["entry_line","exit_line"])["exit_line"].agg([('jishu','count')]).reset_index()
# group_commuting_start.shape

In [12]:
targets = group_commuting_start

In [13]:
targets.head()

,entry_line_id,exit_line_id,jishu
0,4号线,燕房线,6037
1,7号线,S1线,3410
2,8号线,4号线,185476
3,13号线,9号线,63185
4,1号线,8号线,96911


In [14]:
targets["entry_line_id"] = targets["entry_line_id"] + "_in"
targets["exit_line_id"] = targets["exit_line_id"] + "_out"

In [15]:
targets = targets.sort_values(by = "jishu",ascending = False)

In [16]:
targets["cumsum"] = targets["jishu"].cumsum() # 新生成累加列
targets["cum_percent"] = 100*targets["cumsum"]/targets["jishu"].sum()
targets = targets.reset_index(drop=True)

In [28]:
targets[targets["entry_line_id"] == "6号线_in"]

,entry_line_id,exit_line_id,jishu,cumsum,cum_percent
0,6号线_in,6号线_out,1789490,1789490,3.255572
12,6号线_in,10号线_out,541772,11359968,20.666889
18,6号线_in,14号线_out,443400,14289607,25.996704
26,6号线_in,2号线_out,393934,17566550,31.958360
34,6号线_in,4号线_out,328376,20416051,37.142382
37,6号线_in,5号线_out,316348,21380403,38.896802
63,6号线_in,10号线_out,239149,28584004,52.002122
82,6号线_in,1号线_out,214699,32851285,59.765474
87,6号线_in,9号线_out,203128,33879927,61.636855
102,6号线_in,8号线_out,183108,36727332,66.817064


In [18]:
# targets = targets[targets["cum_percent"] <= 100]

In [19]:
targets.shape

(447, 5)

In [20]:
nodes_list_in  = list(targets.entry_line_id.unique())
nodes_list_out = list(targets.exit_line_id.unique())
nodes_list = nodes_list_in + nodes_list_out

In [21]:
len(nodes_list_in)

26

In [22]:
# 生成nodes
nodes = []
# nodes.append({'name':'总支出'})
for i in nodes_list:
    dic = {}
    dic['name'] = i
    nodes.append(dic)

In [23]:
# print(nodes)

In [24]:
# 生成links
links = []
for i in targets.values:
    dic = {}
    dic['source'] = i[0]
    dic['target'] = i[1]
    dic['value'] = i[2]
    links.append(dic)

In [25]:
# print(links)

In [26]:
# # pyecharts 所有方法均支持链式调用。
# c = (Sankey().add("客流量/人",nodes,links,
#             linestyle_opt=opts.LineStyleOpts(width = 50, # 线宽
#                                              opacity=0.2,  # 线透明度
#                                              curve=0.5,  # 线弯曲度，0表示完全不弯曲
#                                              color="target",
#                                              type_="solid" # 'solid', 'dashed', 'dotted'
#                                             ), 
#             label_opts=opts.LabelOpts(position="right",#'top','left','right','bottom','inside','insideLeft'，'insideRight'# 'insideTop'，'insideBottom'， 'insideTopLeft'，'insideBottomLeft'# 'insideTopRight'，'insideBottomRight'
#                                      font_style="italic", # 'normal'，'italic'，'oblique'
#                                       font_size = 14
#                                      ),
#                  ).set_global_opts(title_opts=opts.TitleOpts(title="19年1-6月客流量")))
# # 输出html可视化结果
# c.render('result1.html')

In [27]:
sankey = Sankey("19年9月主要线路客流走向", width=1200, height=800)
sankey.add(
    "sankey",
    nodes,
    links,
    line_opacity=0.2,
    line_curve=0.5,
    line_color="target",
    is_label_show = True,
    is_legend_show = False,
    label_pos="right",
)
sankey.render("19年9月主要线路客流走向100%.html")